## Import's

In [ ]:
import os
import json
import math
from datasets import Dataset, Value, ClassLabel
from keras.utils.np_utils import to_categorical

### Variveis Constantes

In [ ]:
DIR = "../"
dataset_path = "data/"
dataset_twitter_ASBR_path = "datatsets_bases/dataset_twitter_ASBR/"

### Lendo arquivo json

In [ ]:
arquivo_json = None
for arquivo_nome in list(filter(lambda x: x.endswith(".json"), os.listdir(DIR+dataset_path+dataset_twitter_ASBR_path))):
    with open(DIR+dataset_path+dataset_twitter_ASBR_path+arquivo_nome, "r") as arquivo:
        arquivo_json = json.load(arquivo)

In [ ]:
print("Quantidade de dados: " + str(len(arquivo_json)))

In [ ]:
arquivo_json

### Removendo dados repetidos

In [ ]:
dados_nao_repetidos = []
for key in list(arquivo_json.keys()):
    if not arquivo_json[key]["repeat"]:
        dados_nao_repetidos.append(arquivo_json[key])

In [ ]:
dados_nao_repetidos

In [ ]:
print("Qtds de dados retirados: " + str(len(arquivo_json) - len(dados_nao_repetidos)) + " dados retirados, devido serem repetidos!")

In [ ]:
print("Quantidade de dados não repetidos: " + str(len(dados_nao_repetidos)))

### Convertendo para dataset

In [ ]:
dataset = Dataset.from_list(dados_nao_repetidos)

In [ ]:
dataset

### Renomeando as colunas

In [ ]:
dataset.features

In [ ]:
dataset = dataset.rename_column("sentiment", "labels")

In [ ]:
dataset.features

### Removendo as colunas

In [ ]:
dataset.features

In [ ]:
dataset = dataset.remove_columns(["id", "hashtag", "votes", "hard", "group", "repeat"])

In [ ]:
dataset.features

### Arrumando os resultados 


#### Removendo resultado indesejado

In [ ]:
set(dataset["labels"])

In [ ]:
dataset = dataset.filter(lambda x: x["labels"]!="-")

In [ ]:
set(dataset["labels"])

#### Covertendo os resultados para int

In [ ]:
set(dataset["labels"])

In [ ]:
dataset.features

In [ ]:
novo_tipo = dataset.features.copy()
novo_tipo["labels"] = Value("int32")
dataset = dataset.cast(novo_tipo)

In [ ]:
dataset.features

In [ ]:
novo_tipo = dataset.features.copy()
novo_tipo["labels"] = ClassLabel(names=["negative", "neutral", "positive"])
dataset = dataset.cast(novo_tipo)

In [ ]:
dataset.features

In [ ]:
set(dataset["labels"])

#### Retirando valor negativo

In [ ]:
set(dataset["labels"])

In [ ]:
def retira_valor_negativo(data):
    data["labels"] = data["labels"] + 1
    return data

dataset = dataset.map(retira_valor_negativo)

In [ ]:
set(dataset["labels"])

#### Transformando as labels

In [ ]:
set(dataset["labels"])

In [ ]:
num_classes = len(set(dataset["labels"]))
print("Qtd de classes: " + str(num_classes))

In [ ]:
dataset["labels"]

In [ ]:
def label_to_categorical(data):
    data["labels"] = to_categorical(data["labels"], num_classes=num_classes)
    return data

dataset = dataset.map(label_to_categorical)

In [ ]:
dataset["labels"]

### Criar grupos para cross validation

#### Embaralhando os dados

In [ ]:
dataset["text"][:5]

In [ ]:
dataset = dataset.shuffle()

In [ ]:
dataset["text"][:5]

#### Criando lista de grupos

In [ ]:
lista_grupos = list(range(1, 11)) * math.ceil(len(dataset)/10)
lista_grupos = lista_grupos[:len(dataset)]

In [ ]:
len(lista_grupos)

In [ ]:
lista_grupos

In [ ]:
dataset = dataset.add_column("group", lista_grupos)

In [ ]:
dataset.features

In [ ]:
dataset["group"]

#### Verificando os grupos

In [ ]:
def group_by(dataset: Dataset, coluna: str):
    lista_grupos = []
    for grupo in set(dataset[coluna]):
        lista_grupos.append(dataset.filter(lambda x: x[coluna]==grupo))
    return lista_grupos

In [ ]:
lista_datasets = group_by(dataset, "group")

In [ ]:
for index, dataset_part in enumerate(lista_datasets):
    print(str(index+1) + " - " + str(len(dataset_part)))

### Salvando em arquivo .json

In [ ]:
dataset.to_json(DIR+dataset_path+"dataset_twitter_ASBR.json")